In [ ]:
# =========================================================
# --- Opcjonalnie : montowanie Google Drive ---
# =========================================================
# from google.colab import drive
# import os
#
# # Montowanie dysku Google
# drive.mount('/content/drive')
#
# # Ścieżka do folderu roboczego na Drive
# base_export_dir = '/content/drive/MyDrive/colab_cifar10_exports'
# os.makedirs(base_export_dir, exist_ok=True)
#
# print(f"[OK] Google Drive podłączony. Pliki będą zapisywane w:\n{base_export_dir}")

In [1]:
# =========================================================
# --- 0. Import bibliotek ---
# =========================================================
import numpy as np
import matplotlib.pyplot as plt
import datetime

from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import (
    Input,
    Conv2D,
    BatchNormalization,
    Activation,
    MaxPooling2D,
    GlobalAveragePooling2D,
    Flatten,
    Dense,
    Dropout
)
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# =========================================================
# --- 1. Importowanie danych CIFAR10 ---
# =========================================================
# Wczytanie zbioru danych CIFAR10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Wyświetlenie przykładowego obrazu (opcjonalne)
print("Etykieta przykładowego obrazu:", y_train[0])
plt.imshow(x_train[0])
plt.axis('off')
plt.show()

In [ ]:
# =========================================================
# --- 2. Wstępne przetwarzanie danych ---
# =========================================================
# Normalizacja do zakresu [0,1] i dodanie kanału (H,W,1)
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

# One-hot encoding etykiet
num_classes = 10
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat  = to_categorical(y_test, num_classes)

# lista nazw klas CIFAR-10 (użyteczna w wyświetleniach)
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

dodanie sieci BathNormalization dla unikniecia przetrenowania modelu oraz dla stabilizacji gradientu (bo tutaj sa bardzo rozpikselowane obrazy)
po tej zmianie var_loss z 0.93 na 0.83

oryginalne dane:

    model = Sequential([
      oryginalna werjsa
      Input(shape=input_shape),
      Conv2D(32, (3,3), activation='relu'),
      MaxPooling2D((2,2)),
      Conv2D(64, (3,3), activation='relu'),
      MaxPooling2D((2,2)),
      Flatten(),
      Dense(64, activation='relu'),
      Dense(num_classes, activation='softmax')
    ])

In [ ]:
# =========================================================
# --- 3. Definicja modelu sieci neuronowej ---
# =========================================================
input_shape = (32, 32, 3)  # CIFAR-10 to 32x32 RGB

#todo co sie po kolei dzieje wiedziec
model = Sequential([
    Input(shape=input_shape),

    #---------
    Conv2D(32, (3,3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Dropout(0.1), #todo co robi dropout opisac i wiedziec

    #---------
    Conv2D(64, (3,3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2,2)),

    Dropout(0.2),

    #---------

    #siec gesta
    Flatten(),
    Dense(128),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

Zmiana: optimizer = Adam (learing_rate =  1e-2) na learning_rate = 1e-3 -> to zmniejsza niestabilnosc walidacji, program nie "skacze" po wynikach tylko przechodzi powoli

to zmienia oryginalne 1.35 var_loss na 0.93

In [ ]:
# =========================================================
# --- 4. Kompilacja modelu ---
# =========================================================
# Optymalizator Adam, funkcja straty categorical_crossentropy, metryka accuracy
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Wyświetlenie podsumowania modelu
model.summary()

zmiana w EarlyStopping -> monitorujemy val_loss wiec na tym trzeba sie skupic, dodatkowo przywrocenie najlepszych wag na true, oraz zmiana patience na 5

zmiana w ReduceLROnPlateau -> tak samo, monitorowanie val_loss zamiast val_accuracy, zmiana patience na 3

oryginalna wersja (ale po moich zmianach):

    es = EarlyStopping(
      monitor='val_accuracy',   # metryka, którą obserwujemy (np. val_loss, val_accuracy)
      patience=6,               # liczba epok bez poprawy, po których zatrzymujemy trening
      min_delta=1e-5,           # minimalna wymagana zmiana, by uznać, że jest „poprawa”
      mode='max'              # 'min' jeśli monitorujemy straty, 'max' jeśli dokładność
    )
    rlp = ReduceLROnPlateau(
      monitor='val_accuracy',   # metryka do obserwacji
      factor=0.5,           # ile razy zmniejszyć LR (tu: o połowę)
      patience=6,           # liczba epok bez poprawy przed zmniejszeniem LR, rlp.patience < es.patience
      min_delta=1e-5,       # próg czułości jak wyżej
      min_lr=1e-4,          # dolna granica learning rate
      mode='max',           # 'min' dla strat, 'max' dla dokładności
    )

to zmniejsza var_loss z okolo 0.70 na 0.66

In [ ]:
# =========================================================
# --- 5. Trenowanie modelu ---
# =========================================================
es = EarlyStopping(
    monitor='val_loss',   # metryka, którą obserwujemy (np. val_loss, val_accuracy)
    patience=5,               # liczba epok bez poprawy, po których zatrzymujemy trening
    min_delta=1e-5,           # minimalna wymagana zmiana, by uznać, że jest „poprawa”
    mode='min',              # 'min' jeśli monitorujemy straty, 'max' jeśli dokładność
    restore_best_weights=True
)
rlp = ReduceLROnPlateau(
    monitor='val_loss',   # metryka do obserwacji
    factor=0.5,           # ile razy zmniejszyć LR (tu: o połowę)
    patience=3,           # liczba epok bez poprawy przed zmniejszeniem LR, rlp.patience < es.patience
    min_delta=1e-5,       # próg czułości jak wyżej
    min_lr=1e-5,          # dolna granica learning rate
    mode='min',           # 'min' dla strat, 'max' dla dokładności
)

history = model.fit(
    x_train,
    y_train_cat,
    batch_size=64,
    epochs=25,
    validation_data=(x_test, y_test_cat),
    callbacks=[es, rlp],
    verbose=1
)

# Data skończenia treningu - timestamp - znacznik do zapisywania plików
ts = datetime.datetime.now().strftime("_%Y%m%d_%H%M")

In [ ]:
# =========================================================
# --- 6. Ewaluacja modelu ---
# =========================================================
loss, accuracy = model.evaluate(x_test, y_test_cat, verbose=0)
print('Dokładność na zbiorze testowym:', f"{accuracy:.2f}")
print('Strata na zbiorze testowym:', f"{loss:.2f}")

In [ ]:
# =========================================================
# --- 7. Wizualizacja przebiegu treningu (loss) ---
# =========================================================
fig_loss_acc = plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'], label='loss (train)')
plt.plot(history.history['val_loss'], label='loss (val)')
plt.xlabel('Epoka')
plt.ylabel('categorical_crossentropy')
plt.title(f'Strata na zbiorze testowym to {loss:.2f}')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'], label='accuracy (train)')
plt.plot(history.history['val_accuracy'], label='accuracy (val)')
plt.xlabel('Epoka')
plt.ylabel('Accuracy')
plt.title(f"Dokładność na zbiorze testowym to {accuracy:.2f}")
plt.legend()
plt.tight_layout()

# Zapis wykresu
loss_acc_path = os.path.join(base_export_dir, f'training_loss_accuracy{ts}.png')
fig_loss_acc.savefig(loss_acc_path)
print("Zapisano wykres loss/accuracy ->", loss_acc_path)

plt.show()

In [ ]:
# =========================================================
# --- 8. Wizualizacja błędnych klasyfikacji i macierz pomyłek ---
# =========================================================
# Predykcje (etykiety)
pred_probs = model.predict(x_test)
pred_labels = np.argmax(pred_probs, axis=1)
true_labels = np.argmax(y_test_cat, axis=1)  # lub po prostu y_test

# Indeksy błędnych klasyfikacji
incorrect_indices = np.nonzero(pred_labels != true_labels)[0]

# Wyświetlenie kilku błędnych przykładów
n_show = min(3, len(incorrect_indices))
for i in range(n_show):
    idx = incorrect_indices[i]
    plt.figure(figsize=(3,3))
    plt.imshow(x_test[idx].squeeze(), cmap='gray')
    plt.title(f"Prawidłowo: {class_names[true_labels[idx]]}  -> Predykcja: {class_names[pred_labels[idx]]}")
    plt.axis('off')
    plt.show()

# Confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
fig_cm, ax = plt.subplots(figsize=(8,8))
disp.plot(ax=ax, cmap='Blues', colorbar=False)
plt.title('Macierz pomyłek')
# Zapis wykresu
loss_acc_path = os.path.join(base_export_dir, f'confusion_matrix{ts}.png')
fig_cm.savefig(loss_acc_path)
print("Zapisano macierz pomyłek ->", loss_acc_path)
plt.show()
